# 2.リカレントニューラルネットワークスクラッチ

リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。


クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

# 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。


## $$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\
h_t = tanh(a_t)
$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)


$x_{t}$ : 時刻tの入力 (batch_size, n_features)


$W_{x}$ : 入力に対する重み (n_features, n_nodes)


$h_{t-1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


$W_{h}$ : 状態に対する重み。 (n_nodes, n_nodes)


$B$ : バイアス項 (n_nodes,)


初期状態 $h_{0}$ は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

# 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

```python
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
```

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

```python
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```

In [1]:
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [3]:
def forward(x, w_x, w_h, b):
    h_prev = np.zeros([batch_size, n_sequences+1, n_nodes])
    h_tmp = np.zeros([batch_size, n_sequences, n_nodes])
    h_next = np.zeros([batch_size, n_sequences, n_nodes])
    for t in range(n_sequences):
        h_tmp[:, t, :] = np.dot(x[:,t,:], w_x) + np.dot(h_prev[:,t,:], w_h) + b
        h_next[:, t, :] = np.tanh(h_tmp[:, t, :])
        h_prev[:, t+1, :] = h_next[:, t, :]
    
    return h_next, h_prev

In [4]:
h_next, h_prev = forward(x, w_x, w_h, b)

In [5]:
print(h_next)

[[[0.76188798 0.76213958 0.76239095 0.76255841]
  [0.792209   0.8141834  0.83404912 0.84977719]
  [0.79494228 0.81839002 0.83939649 0.85584174]]]


# 【問題3】（アドバンス課題）バックプロパゲーションの実装

バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。


$$
W_x^{\prime} = W_x - \alpha \frac{\partial L}{\partial W_x} \\
$$
$$
W_h^{\prime} = W_h - \alpha \frac{\partial L}{\partial W_h} \\
$$
$$
B^{\prime} = B - \alpha \frac{\partial L}{\partial B}
$$
$$
\alpha :学習率
$$
$$
\frac{\partial L}{\partial W_x} : Whに関する損失Lの勾配
$$
$$
\frac{\partial L}{\partial B}:Bに関する損失Lの勾配
$$
$$
勾配を求めるためのバックプロパゲーションの数式が以下です。
$$
$$
\frac{\partial h_t}{\partial a_t} = \frac{\partial L}{\partial h_t} ×(1-tanh^2(a_t))
$$
$$
\frac{\partial L}{\partial B} = \frac{\partial h_t}{\partial a_t}
$$
$$
\frac{\partial L}{\partial W_x} = x_{t}^{T}\cdot \frac{\partial h_t}{\partial a_t}
$$
$$
\frac{\partial L}{\partial W_h} = h_{t-1}^{T}\cdot \frac{\partial h_t}{\partial a_t}
$$
$$
\frac{\partial L}{\partial h_t}は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。
$$
$$
前の時刻や層に流す誤差の数式は以下です。
$$
$$
\frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_t}{\partial a_t}\cdot W_{h}^{T}
$$
$$
\frac{\partial L}{\partial x_{t}} = \frac{\partial h_t}{\partial a_t}\cdot W_{x}^{T}
$$


In [8]:
def backward(X, w_x, w_h, h_prev, dht):
    """
    バックワード
    Parameters
    ----------
    dA : 次の形のndarray, shape (batch_size, n_nodes2)
        後ろから流れてきた勾配
    Returns
    ----------
    dZ : 次の形のndarray, shape (batch_size, n_nodes1)
        前に流す勾配
    """
    dX = np.zeros([*X.shape])
    dh_next = np.zeros([X.shape[0], X.shape[1]+1, w_x.shape[1]])
    for t in reversed(range(X.shape[1])):
        dh = dht[:, t, :] + dh_next[:, t+1, :]
        dB = np.sum(dh, axis=0)
        dWh = np.dot(h_prev[:, t, :].T, dh)
        dh_next[:, t, :] = np.dot(dh, w_h.T)
        dWx = np.dot(X[:, t, :].T, dh)
        dX[:, t, :] = np.dot(dh, w_x.T)

    return dX

In [9]:
backward(x, w_x, w_h, h_prev, np.arange(12).reshape(1, 3, 4))


array([[[0.657394, 0.86843 ],
        [1.3629  , 1.8783  ],
        [1.62    , 2.27    ]]])